## 3. The Maximum Independent Set (MIS) Problem 

Let's use our recent lessons from preparing checkerboard phases in a square lattice and testing data on hardware to expand our horizon of applications.

Our approach to checkerboard phases, so far, lives within a physicist's life context: we imagine some system, maybe a magnetic compound, that lives in a square lattice. If energetically the local magnetic moments tend to misalign, and if they interact geometrically according to their relative distances, the Rydberg Hamiltonian can predict the lowest-energy excitation patterns of this system. When interactions are strongest only between closest neighbors in the square lattice, the checkerboards pattern arises, and would be known in the magnetism literature as a Néel antiferromagnetic pattern.

But now we will take our physicist's hats off and put a different one. Perhaps that of an applied mathematician, computer scientist, or data scientist. In this case, the same problem above can be approached from a [graph-theory](https://en.wikipedia.org/wiki/Graph_theory) lens, and ultimately be tied to optimization problems of relevance for urban planing, logistics, and much more. 

Here is how we go about reinterpreting the above. We start with a different problem: let's imagine a graph, a collection of vertices, displayed in a square lattice. Let's now assume that these vertices are connected by edges only to their first neighbor, creating a graph in the shape of a square-like net. If we are now tasked with solving the mathematical problem of coloring those vertices (in red, assuming that originally they were white, for concreteness) in a way that two connected vertices cannot be colored at the same time, we are said to be searching for an "independent set" of the graph. Asking for the largest subcollection of colored vertices would bring us the Maximum Independent Set (MIS), which for such a square lattice graph would be nothing but a checkerboards pattern! 

This can be contextualized to several real-world situations. For example, maybe the vertices represent potential geometric positions of a ubiquitous urban asset, perhaps positions for fire hidrants. Edges between those vertices  would suggest that there is some sort of causal relation between those possible positions for fire hidrants. Perhaps placing two of them too close to each other could be a waste of resources or too much redundancy in the system (perhaps except for occasional dogs). Asking for an MIS for this graph would be tantamount to asking to de-facto position those fire hydrants optimally in a square-lattice city in a way that fires can be most efficiently put out, all the while the allocation of resources is most efficient. Same problem!

But not all graph problems of interest live in a square lattice. And connectivities are not always only to the first neighbor. The checkerboards example give us a starting point to notice that analog neutral-atom quantum computers are naturally designed to solve the MIS in different scenarios. So in this section we will learn how to generalize from the square-lattice case, discuss how to statistically explore varied problems, and briefly create an analysis pipeline for data generated from Aquila for MIS applications.



## 3a. MIS, unit-disk graphs

* discuss problem-instance generation: arbitrary graphs vs graphs on a grid. Transition from square-lattice to glassy-systems with defects
* define UDG and why this will be important given the blockade phenomenon (no need to introduce Rb considerations here yet. Leave that to hardware mapping section)

## 3b. Hardware mapping

### i) The blockade radius (dynamic and static?)

The natural analogy between the constraint of the MIS problem and the Rydberg Blockade phenomenon makes mapping of a target problem into quantum hardware a straighforward task. There are, however, some nuances that we need to be aware of. Firstly, edge connectivity of the graphs we can target is limited by the finite character of the Blockade radius $R_{b}$. In the language of graph theory, this means that the subset of problems we can tackle are the so-called Unit Disk Graphs (UDGs).

Determining the $R_b$ scale is somewhat a tricky task. A good practice is to first determine an $R_{min}$, defining the minimal radius in which one would like to have atoms blocking each other, and an $R_{max}$, defining a radius after which one does not want atoms blocking each other. Then, a nice way to fix the desired blockade scale is to take the geometric mean $R_b=\sqrt{R_{min}R_{max}}$. 
For instance, imagine we want the atoms to be connected to their first and second neighbors (when in a square lattice, this is known as a "king's graph", alluding to chess). In this case, $R_{min}=\sqrt{2}a$ -- blocking the first and second neighbors -- and $R_{max}=2a$ -- so not blocking third neighbors and beyond. 

Now that we have defined $R_{b}$, one natural question for the purpose of building the logical graph in a physical device is, how can we tune $R_{b}$ in our quantum device? The relevant observation is that we have two physical "knobs" to tune $R_{b}$: 
$$
R_{b} = \left(\frac{C}{\sqrt{\Omega^{2}+\Delta^{2}}}\right)^{1/6}.
$$
We can leverage this dependence to set our radius $R_{b}$ in terms of a single physical parameter that is tuned during an adiabatic protocol, as outlined below. From a practical standpoint, we choose this parameter to be the detuning such that $R_{b}=\left(\frac{C}{\Delta_{max}}\right)^{1/6}$, where $\Delta_{max}$ is the detuning at the end of the adiabatic protocol.




### ii) Protocol

Now, the question is, in practice, how can we solve the MIS problem on a neutral atom quantum computer? The idea here is to encode the solution to the MIS problem for a given graph in the ground state of the parameterized Hamiltonian of the quantum platform. Thus, in a quantum mechanical framework, MIS is cast into a state preparation problem.
We can rely on a myriad of state preparation protocols, but for the sake of concreteness, here we consider an adiabatic approach where we evolve an easy-to-prepare state in the quantum platform with a slowly-varying Hamiltonian, such that the Hamiltonian at the end of the protocol encodes the target blockade radius of our problem at hand. We remind the reader that the underlying Hamiltonian of our quantum platform is
$$
H=\underbrace{\Omega\sum_{j}\left(|r_{j}\rangle\langle g_{j}|+\text{h.c.}\right)}_{H_{q}}\underbrace{-\Delta\sum_{j}n_{j}+\sum_{i<j}V_{i,j}n_{i}n_{j}}_{H_{c}}
$$


 where to get some intuition on the role of the Hamiltonian parameters, we note that a global large  detuning $\Delta$ 
 to all sites should favor all qubits to be on state $|r\rangle$
, while the Rydberg blockade should do, by itself and without need of our own control, the job of enforcing the constraint of no neighboring 
 excitations. This means that for sufficiently large $\Delta$'s, (but carefully chosen to encode the target Rydberg blockade radius, together with $\Omega$), we expect the expectation value of Rydberg excitations of Hamiltonian's ground state, to be equal to the size of the MIS of the encoded problem.
In the language of classical optimization, this is tantamount to the optimization of the cost function $\langle GS (\Omega,\Theta) | H_{c} |GS(\Omega,\Theta)\rangle$, $|GS(\Omega,\Theta)\rangle$, being the ground state of the target Hamiltonian. 


Thus, we can envision the following protocol: initialize the graph where all the atoms are initialized in the $|g\rangle$, and evolving a very negative global $\Delta$ towards a positive value, with a finite $\Omega$ meanwhile to enable qubit flips from $|g\rangle$ to $|r\rangle$. Since the state we are looking for is classical, we may as well turn $\Omega$ off at the end of the protocol, which also facilitates fixing of the blockade radius: it will simply be defined by $\Delta$  as $R_b = (C_6/\Delta)^{1/6}$, where $C_6=2\pi \times 862690\, \text{MHz}\, \mu \text{m}^6$. The typical choice of $\Omega$ is to make it as large as possible as this defines the characteristic time scale of motion of the system, such that by the end of the protocol, the system undergoes longer time evolution in units of this internal time scale fior a fixed duration of the protocol . Ramping $\Delta$ slowly to its correct final value, then, defines an adiabatic protocol to solve the MIS problem.

In [ ]:
#checking some relevant numbers
print(Rb)
print(RbO)
print(RbD)
print(Delta_end)

7.568067737283432
8.36675464062834
7.568067737283431
28.84624072309878


## 3c. Examples on the defective King's lattice

### i) Problem set-up

We are now in shape to show how to use some of Bloqade's built-in tools to generate defects in a graph and then use Bloqade to solve the Maximal Independent Set (MIS) problem on a Unit Disk Graph (UDG). As a first setp, let us define the real-space layout for the atom array that encodes our problem, which in this case we regard as a square lattice with lattice parameter of 4.5 $\mu$ m and with random vancancies.

In [ ]:
from bloqade.analog import load, save
from bloqade.analog.atom_arrangement import Square
import numpy as np
import os
import matplotlib.pyplot as plt

from bokeh.io import output_notebook
output_notebook() 

if not os.path.isdir("data"):
    os.mkdir("data")

# setting the seed
rng = np.random.default_rng(1234)
a=4.5
lattice =Square(4, lattice_spacing=a).apply_defect_density(0.3, rng=rng).remove_vacant_sites()

#durations = [0.3, 1.6, 0.3]

lattice.show()

Loading BokehJS ...

In this example, we simply call the add_defect_density on the geometry object. The add_defect_density method takes a float between 0 and 1 and uses that as the probability of a site being a defect. The add_defect_count method takes the number of defects to add to the geometry placed in random locations. Both ways take an optional rng argument, a numpy random number generator. If no rng argument is provided, then the default numpy random number generator is used. Using the random number generator allows you to set the seed for reproducibility.

So very well, the atoms in the lattice represent the vertices of a graph. But how do we define edges? As mentioned, two atoms that are too close together are blocked from being both in the $| 1 \rangle$ state. So we will abstract the atoms that are positioned closer than a "blockade radius" distance $R_b$, under which 2 qubits cannot be simultaneously $1$, to be connected by an edge.

Let's imagine we want the atoms to be connected to their first and second neighbors (when in a square lattice, this is known as a "king's graph", alluding to chess). In this case, $R_{min}=\sqrt{2}a$ -- blocking the first and second neighbors -- and $R_{max}=2a$ -- so not blocking third neighbors and beyond. 

Altogether, we want $R_b=\sqrt{2\sqrt{2}}a\approx 7.57\mu m$.

In [ ]:
from bloqade.analog import get_capabilities

# get capabilities for Aquila
aquila_capabilities = get_capabilities()

C6=float(aquila_capabilities.capabilities.rydberg.c6_coefficient)
Omega_max= float(aquila_capabilities.capabilities.rydberg.global_.rabi_frequency_max) #2*np.pi*4 #

Rba=np.sqrt(2*np.sqrt(2))
Rb=Rba*a

Delta_end= (C6)/Rb**6 #final detuning 2*np.pi*11

RbO=(C6/Omega_max)**(1/6)
RbD=(C6/Delta_end)**(1/6)

Now for the protocol

In [101]:
from bloqade.analog import piecewise_linear

T_max=4
sweep_time = 0.8*T_max #time length of the protocol 
ramp_time=0.1*T_max

rabi_amplitude_values = [0.0, Omega_max, Omega_max, 0.0]
detuning_values = [-Delta_end, -Delta_end, Delta_end, Delta_end]
durations = [ramp_time, sweep_time, ramp_time]

RabiProtocol = piecewise_linear(durations,rabi_amplitude_values)
DeltaProtocol = piecewise_linear(durations,detuning_values)

RabiProtocol.show()
DeltaProtocol.show()

run

In [102]:
mis_udg_program=(
        lattice.rydberg.rabi.amplitude.uniform.apply(RabiProtocol)
        .detuning.uniform.apply(DeltaProtocol)
)

result = mis_udg_program.bloqade.python().run(100,interaction_picture=True)

and measure

In [105]:
report = result.report()
report.show()

0.0 13500000.0 0.0 13500000.0


from `report` you should also be able to obtain direclty the bit strings and play with them to update your graphs, etc